# Titanic data EDA and Prediction

**Data description**

Here we have 12 columns-

**PassengerId** : ID of Passenger

**Pclass** : Passenger class (1=1st,2=2nd,3=3rd)

**Survived** : Survival (0=No,1=Yes)

**Sex** : sex(male & female)

**Name** : name of passengers

**Age** : age of passengers

**Sibsp** : Number of Siblings

**Parch** : Number of Parents

**Ticket** : passenger ticket number

**Fare** : Passenger fare(British pound)

**Cabin** : cabin

**Embarked** : Port of Embarkation(C=Cherbourg ,Q=Queenstown, S=Southamption)





In [ ]:
#importing usefull lib

import numpy as np
import pandas as pd
import seaborn as sns

from pandas.plotting import scatter_matrix
import matplotlib.pyplot as plt

In [ ]:
#reading the data using pandas read_csv. 
#It's ofthen used pandas fuction to read csc file.
import pandas as pd
df1=pd.read_csv("../input/titanic/train.csv")
df1.head()

In [ ]:
#checking shape of data frame
df1.shape

In [ ]:
#here we check mean,std,quantiles value using pandas describe function
df1.describe()

In [ ]:
#extracting all columns from the data frame for forother uses
df1.columns.tolist()


In [ ]:
#count NA values
df1.isnull().sum()

In [ ]:
#dropng unrelated column
#here we going to drop cabin bcz it's have lots of nan vales
#here is nothing use in traing og passenger id so simply we drop it using pandas drop()
df=df1.drop(["PassengerId","Ticket","Cabin"],axis=1)
df.head()

In [ ]:
#filing NA values
#here we filling na values by mean() for numerical values 
#and mode() for categorical 
df["Age"].fillna(df["Age"].mean(),inplace=True)
df['Embarked'].fillna(df['Embarked'].mode()[0],inplace=True)

In [ ]:
df.info()

# Exploratory Data Analysis

In [ ]:
#visualisation how many pasanger survived and how many dead
#here we creat a function for bar_chart 
#for avoiding write same code for defrent columns

def bar_chart(column):
    survived=df[df["Survived"]==1][column].value_counts()
    dead=df[df["Survived"]==0][column].value_counts()
    df1=pd.DataFrame([survived,dead])
    df1.index=["Survived","Dead"]
    df1.plot(kind="bar",figsize=(10,5))

In [ ]:
#here we make a bar chart on sex column
#for checking how many male & female
bar_chart("Sex")

**In the above chart we can essly analyse that females have more chance to survived.**

In [ ]:
#here we going to make bar char on Pclass
bar_chart("Pclass")

**by the above chart on Pclass we can say that 1st class passenger have more chance to survived**

In [ ]:
#here we going to make bar chart on sibsp
bar_chart("SibSp")

**by above chart we can analyes that there is more chance to survivrd those who have 0 or 1 siblings**

In [ ]:
bar_chart("Parch")

In [ ]:
bar_chart("Embarked")

**by the above chart we can say that there are more chance to survived for those who bord from Southamption**

**Passenger traveilling from Cherbourg port survived more than other port passenger**

**Dedacting outliers and removing them**

In [ ]:
#visualisation data on boxplot to see the outliers
def box_plot(column):
    df.boxplot(by="Survived",column=[column],grid=True)



In [ ]:
box_plot("Fare")

here we can see outliers above the 100 we can considerd them as outlier

In [ ]:
#checking outliers on Sibsp column
box_plot("SibSp")

by above chart we consider more then 5 siblings as outlierS

In [ ]:
#by value_counts we can see total unique values
df["SibSp"].value_counts()

In [ ]:
#here we chacking largest values row on column Sibsp
df.nlargest(12,["SibSp"])

In [ ]:
#now we gpoing to remove outliers
df=df.drop([159,180,201,324,792,846,863])
df.shape


In [ ]:
#here we going to check outliers on parch
box_plot("Parch")

In [ ]:
df["Parch"].value_counts()

In [ ]:
df.nlargest(12,["Parch"])

In [ ]:
df=df.drop([678])
df.shape


In [ ]:
#here we going to drow heatmap to check co relation between columns 

plt.figure(figsize=(10, 10))
sns.heatmap(df.corr(), annot=True, linewidths=0.05, fmt= '.2f',cmap="magma")
plt.show()

**Frome EDA we got**
* Cabin column have lots of null values so we drop it,
* Ticket and paddenger Id is not usefull and does not have impact on survivl so drop it.
* Passenger travelling in higher class have more chance to survived
* Females survived more then Males.
* In the 1st class Females were more then Males it is also a resion that females have more chance to survived.
* Passenger travelling with siblings ,parents have more chance to survived.
* Passenger traveilling from Cherbourg port survived more than other port passenger.


In [ ]:
#droping an relevant columns
#dividing data X(features) and Y(outcome)
X=df.drop(["Fare","Name","Survived"],axis=True)
y=df["Survived"]

In [ ]:
print(X.shape)
print(y.shape)

Normalize Age columns becose it have high variance

In [ ]:
X["Age"]=np.log(X["Age"])

In [ ]:
#encoding catagorical values
x=pd.get_dummies(X)
x.shape

**Model building**

In [ ]:
#here we going to split data in traing set and testing
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=.25,random_state=0)

**Training LogisticRegression**

In [ ]:
from sklearn.linear_model import LogisticRegression
lr=LogisticRegression(random_state=40)
lr.fit(x_train,y_train)

print(lr.score(x_test,y_test))

**Training DecisionTreeClassifier**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dtc=DecisionTreeClassifier(random_state=140)
dtc.fit(x_train,y_train)

print(dtc.score(x_test,y_test))

**Traing RandomForestClassifier**

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc=RandomForestClassifier(random_state=40,min_impurity_decrease=0.002,min_weight_fraction_leaf=0.001)

rfc.fit(x_train,y_train)

#print(rfc.score(x_test,y_test))

In [ ]:
print(rfc.score(x_test,y_test))

In [ ]:
y_lrpre=lr.predict(x_test)
y_dtc_pre=dtc.predict(x_test)
y_rfc_pre=rfc.predict(x_test)


In [ ]:
from sklearn.metrics import confusion_matrix

def matrix(predection):
    result=confusion_matrix(y_test,predection)
    print(result)

In [ ]:
matrix(y_dtc_pre)

In [ ]:
#Prediction on Testing 

# Prediction on Test data
**clean and feature selection same as training**

In [ ]:
import pandas as pd
import numpy as np
df2=pd.read_csv("../input/titanic/test.csv")

df2.head()

In [ ]:
new_x=df2.drop(["Cabin","PassengerId","Fare","Name","Ticket"],axis=True)
new_x.head()

In [ ]:
new_x.isnull().sum()

In [ ]:
#filing NA values
new_x["Age"].fillna(new_x["Age"].mean(),inplace=True)


In [ ]:
new_x["Age"]=np.log(new_x["Age"])

In [ ]:
new_x=pd.get_dummies(new_x)

In [ ]:
missing_col=set(x_train.columns)-set(new_x.columns)

for col in missing_col:
    new_x[col]=0
    


In [ ]:
new_x=new_x[x_train.columns]
new_x.shape


In [ ]:
#here we used over best train model
new_predict=lr.predict(new_x)
print(new_predict)

In [ ]:
vip=np.array(new_predict).tolist()


In [ ]:
len(vip)

In [ ]:
df2.insert(2,column="Survived",value=vip)
df2.head()

# macking csv(PassengerId & survived) file to upload

In [ ]:
df3=df2.drop(['Pclass','Name','Sex','Age','SibSp','Parch','Ticket','Fare','Cabin','Embarked'],axis=1)
df3.head()

In [ ]:
df3.to_csv('Titanic_modelP_lr.csv',index=False)
df3.head()

In [ ]:
df4=pd.read_csv('Titanic_modelP_lr.csv')
df4.head()

In [ ]:
df4.shape

# Please Up Vote and write suggestion in comments

**Thanks**